<div style="display: flex; align-items: center;">
    <span style="font-size: 24px; color: #422057FF; font-weight: 500;">Index_based Nearest_Neighbour Search</span>
    <img src="logo.svg" style="height: 50px; width: auto; margin-left: auto;"/>
</div>

In [ ]:
import os
import bz2
import time
import glob
import shutil
import subprocess
import pandas as pd

<div style="background-color:#4B6300; color:#F0E5CF; padding: 1px; border-radius: 5px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Make a small subset
 </h2>
</div>

In [ ]:
os.makedirs("simulation_files", exist_ok=True)

# total_data = 'Enamine_REAL_HAC_11_21_666M_CXSMILES.cxsmiles.bz2'
# download from https://enamine.net/compound-collections/real-compounds/real-database

total_data = 'Enamine_REAL_HAC_11_21_1M_CXSMILES.cxsmiles.bz2'
subset_data = 'simulation_files/Enamine_REAL_HAC_11_21_1M_CXSMILES.cxsmiles.bz2'

num_lines = 1000001
with bz2.open(total_data, 'rt') as input_file, bz2.open(subset_data, 'wt') as output_file:
    for i, line in enumerate(input_file):
        if i < num_lines:
            output_file.write(line)
        else:
            break

print("\033[1m\033[34mSubset created successfully\033[0m")

<div style="background-color:#4B6300; color:#F0E5CF; padding: 1px; border-radius: 5px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Convert bz2 to gz format
 </h2>
</div>

In [ ]:
input_file = subset_data
gz_csv = 'simulation_files/Enamine_REAL_HAC_11_21_1M_CXSMILES.gz'

start_time = time.time()

subprocess.run(['spark-submit', '/home/z_main_files/bz2_to_csv.py', input_file, gz_csv], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
os.remove(input_file)

end_time = time.time()
elapsed_time_minutes = round((end_time - start_time) / 60, 2)

os.makedirs("simulation_files", exist_ok=True)
file_path = os.path.join("simulation_files", "bz2_to_csv_time.txt")
with open(file_path, "w") as file:
    file.write(f"{elapsed_time_minutes} minutes")

print(f"\033[1m\033[34mTotal execution time (minutes): \033[91m{elapsed_time_minutes}\033[0m")

<div style="background-color:#4B6300; color:#F0E5CF; padding: 1px; border-radius: 5px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Check the file
 </h2>
</div>

In [ ]:
def process_files(folder_name, file_pattern='*.csv.gz'):
    temp_file_name = folder_name + '1'
    search_pattern = os.path.join(folder_name, file_pattern)
    file_list = glob.glob(search_pattern)
    source_file_path = file_list[0]
    temp_file_path = os.path.join('.', temp_file_name)
    shutil.copy2(source_file_path, temp_file_path)
    shutil.rmtree(folder_name)
    final_file_path = os.path.join('.', folder_name)
    os.rename(temp_file_path, final_file_path)
    return final_file_path

folder_name = gz_csv
final_file_path = process_files(folder_name)
print("\033[1m\033[34mFile processed successfully\033[0m")

In [ ]:
file_path = gz_csv

df_check = pd.read_csv(file_path, compression='gzip', sep=',')
display(df_check.tail())
print(df_check.shape)